In [2]:
import pandas as pd
import json
import seaborn as sns
import networkx as nx
import itertools

In [40]:
def prepare_csv_node_list(data, threshold=0):
    drugs = data['drugName'].value_counts()
    drug = drugs.loc[lambda x: x>threshold]
    conditions = data['condition'].value_counts()
    node_list = [("Id", "label", "type")]
    i=0
    for drug, _ in drugs.items():
        node_list.append((f"drug{i}", drug, "drug"))
        i += 1
    j=0
    for condition,_ in conditions.items():
        node_list.append((f"cond{j}", condition, "cond"))
        j += 1
    return node_list
    
def prepare_csv_edge_list(data, threshold):
    drug_list = df['drugName'].value_counts()
    drug_list = drug_list.loc[lambda x: x>threshold]
    edge_list_repeats = []
    edge_list_singular = []
    print("Progress:\n[", end='')
    for drug, drug_count in drug_list.items():
        # all of the conditions per drug
        conds_series = data.loc[data['drugName'] == drug]['condition'].value_counts()

        for cond, cond_count in conds_series.items():
            # filter df per condition to get df with only the same condition and drug in the loop
            filtered_df = data.loc[(data['drugName'] == drug) & (data['condition'] == cond)]
            # for the long version, iterate over all instances
            for index, row in filtered_df.iterrows():
                edge_list_repeats.append((row["drugName"], row["condition"], row["rating"], row["usefulCount"]))       
            sum = filtered_df['rating'].sum()
            avg = sum / filtered_df.shape[0]
            # short version with no repeats
            edge_list_singular.append((drug, cond, cond_count, avg))
        print(".", end='')
    print("]\nDone!")
    return (edge_list_singular, edge_list_repeats)

In [3]:
with open('train.jsonl') as f:
    df_raw = pd.DataFrame(json.loads(line) for line in f)
df_raw.drop('Unnamed: 0', axis=1, inplace=True)
df = df_raw.drop(df_raw[df_raw['condition'].str.contains("<", na=False)].index)
print(f"removed {df_raw.shape[0]-df.shape[0]} rows")
df

removed 900 rows


,drugName,condition,review,rating,date,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37
...,...,...,...,...,...,...
161292,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0,"May 31, 2015",125
161293,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,"November 1, 2011",34
161294,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0,"March 15, 2014",35
161295,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10.0,"September 19, 2015",79


In [54]:
ranked_drugs = df['drugName'].value_counts()
#conditions = df['condition'].value_counts()
# filter above certain threshhold 
#thresh_ranked_drugs = ranked_drugs.loc[lambda x: x>100]
ranked_drugs.loc[lambda x: x>200]

Levonorgestrel                       3657
Etonogestrel                         3336
Ethinyl estradiol / norethindrone    2850
Nexplanon                            2149
Ethinyl estradiol / norgestimate     2117
                                     ... 
Ondansetron                           203
Lutera                                203
Seasonique                            203
Suvorexant                            201
Wellbutrin XL                         201
Name: drugName, Length: 179, dtype: int64

In [44]:
node_list = prepare_csv_node_list(df, 500)
print(len(node_list))
for i in range(30):
    print(node_list[i])

4229
('Id', 'label', 'type')
('drug0', 'Levonorgestrel', 'drug')
('drug1', 'Etonogestrel', 'drug')
('drug2', 'Ethinyl estradiol / norethindrone', 'drug')
('drug3', 'Nexplanon', 'drug')
('drug4', 'Ethinyl estradiol / norgestimate', 'drug')
('drug5', 'Ethinyl estradiol / levonorgestrel', 'drug')
('drug6', 'Phentermine', 'drug')
('drug7', 'Sertraline', 'drug')
('drug8', 'Escitalopram', 'drug')
('drug9', 'Mirena', 'drug')
('drug10', 'Implanon', 'drug')
('drug11', 'Gabapentin', 'drug')
('drug12', 'Bupropion', 'drug')
('drug13', 'Venlafaxine', 'drug')
('drug14', 'Miconazole', 'drug')
('drug15', 'Medroxyprogesterone', 'drug')
('drug16', 'Citalopram', 'drug')
('drug17', 'Bupropion / naltrexone', 'drug')
('drug18', 'Lexapro', 'drug')
('drug19', 'Duloxetine', 'drug')
('drug20', 'Metronidazole', 'drug')
('drug21', 'Contrave', 'drug')
('drug22', 'Drospirenone / ethinyl estradiol', 'drug')
('drug23', 'Cymbalta', 'drug')
('drug24', 'Depo-Provera', 'drug')
('drug25', 'Quetiapine', 'drug')
('drug26', 

In [ ]:
# Takes a while
edge_list_repeats = []
edge_list_singular = []
print("Progress:")
for drug, drug_count in ranked_drugs.items():
    # all of the conditions per drug
    conds_series = df.loc[df['drugName'] == drug]['condition'].value_counts()
    
    for cond, cond_count in conds_series.items():
        # filter df per condition to get df with only the same condition and drug in the loop
        
        filtered_df = df.loc[(df['drugName'] == drug) & (df['condition'] == cond)]
        for index, row in filtered_df.iterrows():
            edge_list_repeats.append((row["drugName"], row["condition"], row["rating"], row["usefulCount"]))
        sum = filtered_df['rating'].sum()
        avg = sum / filtered_df.shape[0]
        
        edge_list_singular.append((drug, cond, cond_count, avg))
    print(".", end='')
print("Done!")

In [31]:
filtered_csv_list_short, filtered_csv_list_long = prepare_csv_list(df, 200)

Progress:
[.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [33]:
len(filtered_csv_list_long)

159498

In [23]:
csv_list_long = [("Id", "drug", "condition", "rating", "usefulCount")]
csv_list_short = [("Id", "drug", "condition", "numOfInstances", "averageRating")]
for i, row in enumerate(edge_list_repeats):
    csv_list_long.append((i, row[0], row[1], row[2], row[3]))
for i, row in enumerate(edge_list_singular):
    csv_list_short.append((i, row[0], row[1], row[2], row[3]))

In [24]:
for i in range(20):
    print(csv_list_long[i])
for i in range(20):
    print(csv_list_short[i])

('Id', 'drug', 'condition', 'rating', 'usefulCount')
(0, 'Levonorgestrel', 'Birth Control', 1.0, 10)
(1, 'Levonorgestrel', 'Birth Control', 10.0, 1)
(2, 'Levonorgestrel', 'Birth Control', 2.0, 2)
(3, 'Levonorgestrel', 'Birth Control', 3.0, 4)
(4, 'Levonorgestrel', 'Birth Control', 10.0, 1)
(5, 'Levonorgestrel', 'Birth Control', 4.0, 3)
(6, 'Levonorgestrel', 'Birth Control', 10.0, 2)
(7, 'Levonorgestrel', 'Birth Control', 7.0, 1)
(8, 'Levonorgestrel', 'Birth Control', 9.0, 2)
(9, 'Levonorgestrel', 'Birth Control', 5.0, 4)
(10, 'Levonorgestrel', 'Birth Control', 6.0, 6)
(11, 'Levonorgestrel', 'Birth Control', 6.0, 7)
(12, 'Levonorgestrel', 'Birth Control', 10.0, 2)
(13, 'Levonorgestrel', 'Birth Control', 10.0, 11)
(14, 'Levonorgestrel', 'Birth Control', 8.0, 3)
(15, 'Levonorgestrel', 'Birth Control', 1.0, 1)
(16, 'Levonorgestrel', 'Birth Control', 1.0, 4)
(17, 'Levonorgestrel', 'Birth Control', 8.0, 21)
(18, 'Levonorgestrel', 'Birth Control', 6.0, 13)
('Id', 'drug', 'condition', 'numOfIn

In [25]:
import csv
with open('gephi_long_1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(csv_list_long)
with open('gephi_short_1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(csv_list_short)


In [22]:
import json
graph_data = {
    'drug' : ranked_drugs.axes[0].values.tolist(),
    'condition' : conditions.axes[0].values.tolist(),
    'edges' : edge_list
}
with open('graph_dict.json', 'w') as f:
    f.write(json.dumps(graph_data))

NameError: name 'edge_list' is not defined

In [ ]:
import csv
for edge in edge_l 